https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a

In [1]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)

In [2]:
import pandas as pd

In [3]:
file = "./houseprice_data/train.csv"

df = sqlContext.read.format('csv').options(header = 'true',nanValue = 'NA',nullValue = 'NA',inferSchema = True).load(file)

In [6]:
df.take(1)

[Row(Id=1, MSSubClass=60, MSZoning='RL', LotFrontage=65, LotArea=8450, Street='Pave', Alley=None, LotShape='Reg', LandContour='Lvl', Utilities='AllPub', LotConfig='Inside', LandSlope='Gtl', Neighborhood='CollgCr', Condition1='Norm', Condition2='Norm', BldgType='1Fam', HouseStyle='2Story', OverallQual=7, OverallCond=5, YearBuilt=2003, YearRemodAdd=2003, RoofStyle='Gable', RoofMatl='CompShg', Exterior1st='VinylSd', Exterior2nd='VinylSd', MasVnrType='BrkFace', MasVnrArea=196, ExterQual='Gd', ExterCond='TA', Foundation='PConc', BsmtQual='Gd', BsmtCond='TA', BsmtExposure='No', BsmtFinType1='GLQ', BsmtFinSF1=706, BsmtFinType2='Unf', BsmtFinSF2=0, BsmtUnfSF=150, TotalBsmtSF=856, Heating='GasA', HeatingQC='Ex', CentralAir='Y', Electrical='SBrkr', 1stFlrSF=856, 2ndFlrSF=854, LowQualFinSF=0, GrLivArea=1710, BsmtFullBath=1, BsmtHalfBath=0, FullBath=2, HalfBath=1, BedroomAbvGr=3, KitchenAbvGr=1, KitchenQual='Gd', TotRmsAbvGrd=8, Functional='Typ', Fireplaces=0, FireplaceQu=None, GarageType='Attchd'

In [7]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: integer (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |-

In [8]:
df.describe().toPandas().round(2)

,summary,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,count,1460,1460,1460,1201,1460,1460,91,1460,1460,...,1460,7,281,54,1460,1460,1460,1460,1460,1460
1,mean,730.5,56.897260273972606,None,70.04995836802665,10516.828082191782,None,None,None,None,...,2.758904109589041,None,None,None,43.489041095890414,6.321917808219178,2007.8157534246575,None,None,180921.19589041095
2,stddev,421.6100093688479,42.30057099381045,None,24.28475177448321,9981.26493237915,None,None,None,None,...,40.17730694453021,None,None,None,496.1230244579441,2.7036262083595113,1.3280951205521145,None,None,79442.50288288663
3,min,1,20,C (all),21,1300,Grvl,Grvl,IR1,Bnk,...,0,Ex,GdPrv,Gar2,0,1,2006,COD,Abnorml,34900
4,max,1460,190,RM,313,215245,Pave,Pave,Reg,Lvl,...,738,Gd,MnWw,TenC,15500,12,2010,WD,Partial,755000


In [9]:
import pandas as pd 
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int' or t[1] == 'double']
sample_data = df.select(numeric_features).sample(False,0.8).toPandas()
sample_data.shape

(1170, 38)

In [10]:
ax = pd.scatter_matrix(sample_data,figsize=(10,10))
n = len(sample_data.columns)
for i in range(n):
    v = ax[i,0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = ax[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

C:\Users\ko_kaku\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.scatter_matrix is deprecated, use pandas.plotting.scatter_matrix instead
  """Entry point for launching an IPython kernel.


In [11]:
sample_data = sample_data.dropna().any()

In [12]:
df = df.fillna(0)
df.show()

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

In [13]:
import six
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to SalePrice for ", i, df.stat.corr('SalePrice',i))

Correlation to SalePrice for  Id -0.021916719443430967
Correlation to SalePrice for  MSSubClass -0.08428413512659509
Correlation to SalePrice for  LotFrontage 0.2096239447994837
Correlation to SalePrice for  LotArea 0.2638433538714051


IllegalArgumentException: 'requirement failed: Currently correlation calculation for columns with dataType StringType not supported.'

In [14]:
numeric_features.remove('SalePrice')
numeric_features

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [15]:
from pyspark.ml.feature import VectorAssembler
features = ['YearBuilt','1stFlrSF','BedroomAbvGr']
vectorAssembler  = VectorAssembler(inputCols = numeric_features,outputCol = 'features')

v_df =vectorAssembler.transform(df)
v_df =v_df.select('features','SalePrice')
v_df.show()

+--------------------+---------+
|            features|SalePrice|
+--------------------+---------+
|[1.0,60.0,65.0,84...|   208500|
|[2.0,20.0,80.0,96...|   181500|
|[3.0,60.0,68.0,11...|   223500|
|[4.0,70.0,60.0,95...|   140000|
|[5.0,60.0,84.0,14...|   250000|
|[6.0,50.0,85.0,14...|   143000|
|[7.0,20.0,75.0,10...|   307000|
|[8.0,60.0,0.0,103...|   200000|
|[9.0,50.0,51.0,61...|   129900|
|[10.0,190.0,50.0,...|   118000|
|(37,[0,1,2,3,4,5,...|   129500|
|[12.0,60.0,85.0,1...|   345000|
|[13.0,20.0,0.0,12...|   144000|
|[14.0,20.0,91.0,1...|   279500|
|[15.0,20.0,0.0,10...|   157000|
|(37,[0,1,2,3,4,5,...|   132000|
|[17.0,20.0,0.0,11...|   149000|
|(37,[0,1,2,3,4,5,...|    90000|
|[19.0,20.0,66.0,1...|   159000|
|(37,[0,1,2,3,4,5,...|   139000|
+--------------------+---------+
only showing top 20 rows



In [16]:
splits = v_df.randomSplit([0.7,0.3])
train_df = splits[0]
test_df = splits[1]

In [17]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features',labelCol = 'SalePrice')
lr_model = lr.fit(train_df)

In [18]:
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-1.127134885175422,-166.8373679173135,0.5058915427399574,0.40693264880867824,17141.196054990294,4624.0237404771315,296.17371475794596,78.63646531007903,34.05356026188821,8.369123632128499,-1.9902416113121308,-1.6945794732207402,6.940887054950831,26.584170356030207,25.100910662208687,-0.9189787825973561,30.6641316037147,10851.418404046482,1169.6886439301443,2809.517208161788,-3645.9839447317004,-12289.10281958874,-23458.959526986313,5937.131248345719,1046.1834838785123,-17.396930558474335,20616.45280028966,-12.282763827011454,31.601382305003956,4.989827490680489,6.098575632502276,20.41491629723609,54.026521140824826,-78.19327087449618,0.7384338356800483,53.62090619041027,-1019.1732563790704]
Intercept: 1274886.2966385938


In [19]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 34914.498321
r2: 0.821333


In [20]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation  import RegressionEvaluator

rfr = RandomForestRegressor(featuresCol='features',labelCol = 'SalePrice')
rfr_model = rfr.fit(train_df)

In [21]:
predictions = rfr_model.transform(test_df)

In [22]:
evaluator = RegressionEvaluator(
    labelCol="SalePrice", predictionCol="prediction", metricName="rmse")


In [23]:
rmse = evaluator.evaluate(predictions)

In [24]:
rmse

32727.055263845676

In [ ]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .config("spark.jars.packages", "Azure:mmlspark:0.15") \
            .getOrCreate()


In [ ]:
import mmlspark